In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [3]:
%%sql

DROP TABLE IF EXISTS COURSE_CAT_DIMENSION;
CREATE TABLE COURSE_CAT_DIMENSION (
    cat_id INTEGER NOT NULL PRIMARY KEY,
    catalog_id TEXT,
    catalog_year INTEGER,
    program_id INTEGER NOT NULL,
    course_title TEXT NOT NULL,
    credits INTEGER NOT NULL,
    prereqs TEXT,
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,
    description TEXT);
DROP TABLE IF EXISTS PROGRAMS_DIMENSION;
CREATE TABLE PROGRAMS_DIMENSION(
    program_id INTEGER NOT NULL PRIMARY KEY,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL);
DROP TABLE IF EXISTS COURSES_DIMENSION;
CREATE TABLE COURSES_DIMENSION(
    course_id INTEGER NOT NULL PRIMARY KEY,
    crn TEXT,
    catalog_id TEXT,
    section INTEGER);
DROP TABLE IF EXISTS LOCATION_DIMENSION;
CREATE TABLE LOCATION_DIMENSION(
    location_id INTEGER NOT NULL PRIMARY KEY,
    location TEXT);
DROP TABLE IF EXISTS PROFESSOR_DIMENSION;
CREATE TABLE PROFESSOR_DIMENSION(
    professor_id INTEGER NOT NULL PRIMARY KEY,
    name TEXT NOT NULL);
DROP TABLE IF EXISTS COURSE_MEETINGS_FACTS;
CREATE TABLE COURSE_MEETINGS_FACTS (
    coursemeeting_id INTEGER NOT NULL PRIMARY KEY,
    cat_id INTEGER,
    course_id INTEGER,
    location_id INTEGER,
    program_id INTEGER,
    professor_id TEXT,
    day TEXT,
    start TEXT,
    end TEXT,
    term TEXT,
    cap INTEGER,
    act INTEGER,
    rem INTEGER,
    FOREIGN KEY (cat_id) REFERENCES COURSE_CAT_DIMENSION (cat_id),
    FOREIGN KEY (course_id) REFERENCES COURSES_DIMENSION (course_id),
    FOREIGN KEY (location_id) REFERENCES LOCATION_DIMENSION (location_id),
    FOREIGN KEY (program_id) REFERENCES PROGRAMS_DIMENSION (program_id)
    FOREIGN KEY (professor_id) REFERENCES PROFESSOR_DIMENSION (professor_id));


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [4]:
%%sql
ATTACH DATABASE 'CourseData.db' as CourseData;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [5]:
%%sql
DELETE FROM PROGRAMS_DIMENSION;
INSERT INTO PROGRAMS_DIMENSION(program_id,program_code,program_name)
    SELECT DISTINCT PROGRAMS.program_id, PROGRAMS.program_code, PROGRAMS.program_name
    FROM PROGRAMS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
83 rows affected.


[]

In [6]:
%%sql
DELETE FROM PROFESSOR_DIMENSION;
INSERT INTO PROFESSOR_DIMENSION(professor_id,name)
    SELECT DISTINCT PROFESSORS.professor_id,PROFESSORS.name
    FROM PROFESSORS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
1095 rows affected.


[]

In [7]:
%%sql
DELETE FROM LOCATION_DIMENSION;
INSERT INTO LOCATION_DIMENSION (location)
    SELECT DISTINCT COURSE_MEETINGS.location
    FROM COURSE_MEETINGS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
207 rows affected.


[]

In [8]:
%%sql
DELETE FROM COURSES_DIMENSION;
INSERT INTO COURSES_DIMENSION(course_id,crn,catalog_id,section)
    SELECT DISTINCT COURSES.course_id,COURSES.crn,COURSES.catalog_id,COURSES.section
    FROM COURSES;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
15937 rows affected.


[]

In [9]:
%%sql
DELETE FROM COURSE_CAT_DIMENSION;
INSERT INTO COURSE_CAT_DIMENSION(cat_id,catalog_id,catalog_year,program_id,course_title,credits,prereqs,coreqs,fees,attributes,description)
    SELECT DISTINCT C.cat_id,C.catalog_id,C.catalog_year,C.program_id,C.course_title,C.credits,C.prereqs,C.coreqs,C.fees,C.attributes,C.description
    FROM COURSE_CATALOG AS C;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
4440 rows affected.


[]

In [ ]:
%%sql
DELETE FROM COURSE_MEETING_FACTS;
INSERT INTO COURSE_MEETING_FACTS (cat_id,course_id,location_id,program_id,professor_id,day,start,end,term,cap,act,rem)
    SELECT DISTINCT C.cat_id,C1.course_id,L.location_id,P.program_id,P1.professor_id,day,start,end,term,cap,act,rem
    FROM COURSE_MEETINGS
    LEFT JOIN COURSE_CAT_DIMENSION AS C

In [10]:
%%sql
DETACH DATABASE 'CourseData';

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [11]:
%%sql
vacuum;

 * sqlite:///CourseDataWarehouse.db
Done.


[]